# Part 1: Initial EDA and benchmarking 
## Dataloading 
  
Covering: 
  - one-hot encoding
  - sparse tensors 
  - making data objects for PyG
  - train/val/test splits


In [ ]:
cd drive/MyDrive/GNN_in_action/VGAE


/content/drive/MyDrive/GNN_in_action/VGAE


In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


1.12.1+cu113


In [ ]:
import numpy as np
import glob

In [ ]:
filename = 'data/new_AMZN_electronics.npz'

In [ ]:
#get data (where this has already been pre-processed to show how to convert to sparse matrix)
data = np.load(filename)

loader = dict(data)
print(loader)

adj_matrix = torch.tensor(loader['adj_data'])
feature_matrix = torch.tensor(loader['attr_data'])
labels = loader['labels']

class_names = loader.get('class_names')
metadata = loader.get('metadata')

{'adj_data': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'attr_data': array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32), 'labels': array([6, 4, 3, ..., 1, 2, 3]), 'class_names': array(['Film Photography', 'Digital Cameras', 'Binoculars & Scopes',
       'Lenses', 'Tripods & Monopods', 'Video Surveillance',
       'Lighting & Studio', 'Flashes'], dtype='<U19')}


In [ ]:

[print(i, c) for i,c in zip(list(loader.get('class_names')) , np.unique(loader.get('labels')))]

Film Photography 0
Digital Cameras 1
Binoculars & Scopes 2
Lenses 3
Tripods & Monopods 4
Video Surveillance 5
Lighting & Studio 6
Flashes 7


[None, None, None, None, None, None, None, None]

In [ ]:
print(f'Number of items = {feature_matrix.shape[0]}, Number of features = {feature_matrix.shape[1]}')

print(f'Number of items = {adj_matrix.shape[0]} connected to {adj_matrix.shape[1]} items')

print(f'Number of items = {feature_matrix.shape[0]}, Number of labels = {labels.shape[0]}')

Number of items = 7650, Number of features = 745
Number of items = 7650 connected to 7650 items
Number of items = 7650, Number of labels = 7650


In [ ]:
#Example of one-hot encoding of text features in the data for node 0
feature_matrix[0,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 

In [ ]:
# from torch_sparse import SparseTensor


In [ ]:
from torch_sparse import SparseTensor
# Convert to sparse matrix
edge_index = adj_matrix.nonzero(as_tuple=False).t()
edge_weight = adj_matrix[edge_index[0], edge_index[1]]
num_nodes = len(labels)

adj = SparseTensor(row=edge_index[0], col=edge_index[1],
                   sparse_sizes=(num_nodes, num_nodes))



In [ ]:
edge_index[1]

tensor([1210, 2198, 2239,  ..., 2239, 3295, 4196])

In [ ]:
#construct data object

from torch_geometric.data import Data

data = Data(x=feature_matrix, y=labels, adj_t=adj)

data

Data(x=[7650, 745], y=[7650], adj_t=[7650, 7650, nnz=143663])

In [ ]:
# Split (for node prediction)

from torch_geometric.transforms import RandomNodeSplit, RandomLinkSplit

transform = RandomLinkSplit()
transform(data)

(Data(x=[7650, 745], y=[7650], adj_t=[7650, 7650, nnz=143663], edge_index=[2, 100565], edge_label=[201130], edge_label_index=[2, 201130]),
 Data(x=[7650, 745], y=[7650], adj_t=[7650, 7650, nnz=143663], edge_index=[2, 100565], edge_label=[28732], edge_label_index=[2, 28732]),
 Data(x=[7650, 745], y=[7650], adj_t=[7650, 7650, nnz=143663], edge_index=[2, 114931], edge_label=[57464], edge_label_index=[2, 57464]))

In [ ]:
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling

In [ ]:
from torch_geometric.nn import GAE, VGAE, GCNConv

class GCNEncoder(torch.nn.Module):
    def __init__(self, input_size, layers, latent_dims):
        super().__init__()
        self.layer0 = GCNConv(input_size, layers[0])
        self.layer1 = GCNConv(layers[0], layers[1])
        self.layer2 = GCNConv(layers[1], latent_dims)

    def forward(self, x, edge_index):
        x = self.layer0(x, edge_index).relu()
        x = self.layer1(x, edge_index).relu()
        return self.layer2(x, edge_index)

class InnerProductDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
  
    def forward(self, z, edge_index):
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value)

class GraphAutoEncoder(torch.nn.Module):
    def __init__(self, input_size, layers, latent_dims):
        super().__init__()
        self.encoder = GCNEncoder(input_size, layers, latent_dims)
        self.decoder = InnerProductDecoder()

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
class VariationalGCNEncoder(torch.nn.Module):
  def __init__(self, input_size, layers, latent_dims):
    super().__init__()
    self.layer0 = GCNConv(input_size, layers[0])
    self.layer1 = GCNConv(layers[0], layers[1])
    self.mu = GCNConv(layers[1], latent_dims)
    self.logvar = GCNConv(layers[1], latent_dims)
    
  def forward(self, x, edge_index):
    x = self.layer0(x, edge_index).relu()
    x = self.layer1(x, edge_index).relu()
    mu = self.mu(x, edge_index)
    logvar = self.logvar(x, edge_index)
    if self.train():
       return mu, logvar

class InnerProductDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
  
    def forward(self, z, edge_index):
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value)

class GraphAutoEncoder(torch.nn.Module):
    def __init__(self, input_size, layers, latent_dims):
        super().__init__()
        self.encoder = GCNEncoder(input_size, layers, latent_dims)
        self.decoder = InnerProductDecoder()

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:

class GCNEncoder(torch.nn.Module): #B
    def __init__(self, input_size, layers, latent_dim): #B
        super().__init__()
        self.conv0 = GCNConv(input_size, layers[0]) #C
        self.conv1 = GCNConv(layers[0], layers[1]) #C
        self.conv2 = GCNConv(layers[1], latent_dim) #C

    def forward(self, x, edge_index): #D
        x = self.conv0(x, edge_index).relu()#D
        x = self.conv1(x, edge_index).relu()#D
        return self.conv2(x, edge_index) #D


class VariationalGCNEncoder(torch.nn.Module): #A
  def __init__(self, input_size, layers, latent_dims):
    super().__init__()
    self.layer0 = GCNConv(input_size, layers[0])
    self.layer1 = GCNConv(layers[0], layers[1])
    self.mu = GCNConv(layers[1], latent_dims)
    self.logvar = GCNConv(layers[1], latent_dims)
    
  def forward(self, x, edge_index):
    x = self.layer0(x, edge_index).relu()
    x = self.layer1(x, edge_index).relu()
    mu = self.mu(x, edge_index)
    logvar = self.logvar(x, edge_index)
    return mu, logvar



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [ ]:
train_data, val_data, test_data = transform(data)

In [ ]:
def train(model, criterion, optimizer):
    
    model.train() 
  
    optimizer.zero_grad() 
    z = model.encode(train_data.x, train_data.edge_index) 

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')
    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1) 
    loss = criterion(out, edge_label) + model.kl_loss() 
    loss.backward()
    optimizer.step()

    return loss
        

In [ ]:
num_features = feature_matrix.shape[1]
variational = True

input_size, latent_dims = num_features, 16
layers = [512, 256]
if not variational:
  model = GAE(GCNEncoder(input_size, layers, latent_dims))
else:
  model = VGAE(VariationalGCNEncoder(input_size, layers, latent_dims))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


In [ ]:

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

                                                          # |                 

In [ ]:
print(linear, variational)
best_val_auc = final_test_auc = 0
for epoch in range(1, 201):
    loss = train(model, criterion,optimizer)
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')



True True
Epoch: 001, Loss: 0.7414, Val: 0.8177, Test: 0.8137
Epoch: 002, Loss: 0.9697, Val: 0.8148, Test: 0.8102
Epoch: 003, Loss: 0.7447, Val: 0.8027, Test: 0.7986
Epoch: 004, Loss: 0.7442, Val: 0.7947, Test: 0.7899
Epoch: 005, Loss: 0.7456, Val: 0.3467, Test: 0.3485
Epoch: 006, Loss: 0.7465, Val: 0.1803, Test: 0.1843
Epoch: 007, Loss: 0.7461, Val: 0.1856, Test: 0.1898
Epoch: 008, Loss: 0.7457, Val: 0.2640, Test: 0.2734
Epoch: 009, Loss: 0.7451, Val: 0.6866, Test: 0.6878
Epoch: 010, Loss: 0.7453, Val: 0.7579, Test: 0.7552
Epoch: 011, Loss: 0.7453, Val: 0.7621, Test: 0.7586
Epoch: 012, Loss: 0.7437, Val: 0.7324, Test: 0.7331
Epoch: 013, Loss: 0.7440, Val: 0.6337, Test: 0.6350
Epoch: 014, Loss: 0.7434, Val: 0.4408, Test: 0.4467
Epoch: 015, Loss: 0.7424, Val: 0.3072, Test: 0.3138
Epoch: 016, Loss: 0.7436, Val: 0.2974, Test: 0.3045
Epoch: 017, Loss: 0.7435, Val: 0.3986, Test: 0.4044
Epoch: 018, Loss: 0.7428, Val: 0.5704, Test: 0.5735
Epoch: 019, Loss: 0.7430, Val: 0.6895, Test: 0.6919
Ep